#### MODEL TRAINING

##### 1.1 Import Data and Required Packages

In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, precision_score, recall_score, f1_score, roc_auc_score,roc_curve 

from sklearn.decomposition import PCA
import warnings
import scipy

In [2]:
df=pd.read_csv('updated_data.csv')

In [3]:

df.shape
df.drop(columns=["ApplicationDate"], axis=1, inplace=True)

In [20]:
X=df.drop(columns=['LoanApproved'], axis=1)
y=df['LoanApproved']
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30440 entries, 0 to 30439
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         30440 non-null  int64  
 1   AnnualIncome                30440 non-null  int64  
 2   CreditScore                 30440 non-null  int64  
 3   EmploymentStatus            30440 non-null  object 
 4   EducationLevel              30440 non-null  object 
 5   LoanAmount                  30440 non-null  int64  
 6   LoanDuration                30440 non-null  int64  
 7   MaritalStatus               30440 non-null  object 
 8   NumberOfDependents          30440 non-null  int64  
 9   HomeOwnershipStatus         30440 non-null  object 
 10  MonthlyDebtPayments         30440 non-null  int64  
 11  CreditCardUtilizationRate   30440 non-null  float64
 12  NumberOfOpenCreditLines     30440 non-null  int64  
 13  NumberOfCreditInquiries     304

#### Train Test Split Model Training

In [5]:
# separate dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((24352, 31), (6088, 31))

In [6]:
#create column transformer 
num_features=X.select_dtypes(exclude="object").columns
cat_features=X.select_dtypes(include="object").columns

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer,cat_features),
        ("StandardScaler", numeric_transformer, num_features),

    ]
)
num_features, cat_features


(Index(['Age', 'AnnualIncome', 'CreditScore', 'LoanAmount', 'LoanDuration',
        'NumberOfDependents', 'MonthlyDebtPayments',
        'CreditCardUtilizationRate', 'NumberOfOpenCreditLines',
        'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'BankruptcyHistory',
        'PreviousLoanDefaults', 'PaymentHistory', 'LengthOfCreditHistory',
        'SavingsAccountBalance', 'CheckingAccountBalance', 'TotalAssets',
        'TotalLiabilities', 'UtilityBillsPaymentHistory', 'JobTenure',
        'BaseInterestRate', 'InterestRate', 'MonthlyLoanPayment',
        'TotalDebtToIncomeRatio', 'RiskScore'],
       dtype='object'),
 Index(['EmploymentStatus', 'EducationLevel', 'MaritalStatus',
        'HomeOwnershipStatus', 'LoanPurpose'],
       dtype='object'))

In [7]:
preprocessor

ColumnTransformer(transformers=[('OneHotEncoder', OneHotEncoder(),
                                 Index(['EmploymentStatus', 'EducationLevel', 'MaritalStatus',
       'HomeOwnershipStatus', 'LoanPurpose'],
      dtype='object')),
                                ('StandardScaler', StandardScaler(),
                                 Index(['Age', 'AnnualIncome', 'CreditScore', 'LoanAmount', 'LoanDuration',
       'NumberOfDependents', 'MonthlyDebtPayments',
       'CreditCardUtilizationRa...
       'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'BankruptcyHistory',
       'PreviousLoanDefaults', 'PaymentHistory', 'LengthOfCreditHistory',
       'SavingsAccountBalance', 'CheckingAccountBalance', 'TotalAssets',
       'TotalLiabilities', 'UtilityBillsPaymentHistory', 'JobTenure',
       'BaseInterestRate', 'InterestRate', 'MonthlyLoanPayment',
       'TotalDebtToIncomeRatio', 'RiskScore'],
      dtype='object'))])

In [8]:
X_train = preprocessor.fit_transform(X_train)

# Convert to dense array if it's sparse
if isinstance(X_train, (scipy.sparse.csr_matrix, scipy.sparse.csc_matrix)):
    X_train = X_train.toarray()

# X_transformed is now the preprocessed data in array format
print(pd.DataFrame(X_train))

        0    1    2    3    4    5    6    7    8    9   ...        37  \
0      1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  ... -0.572375   
1      1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  ...  2.303827   
2      1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  ... -0.014812   
3      1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  ... -0.264132   
4      1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  ...  0.647845   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...       ...   
24347  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  ... -0.265012   
24348  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  ... -0.668234   
24349  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.610908   
24350  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  ... -0.350757   
24351  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  ... -0.565780   

             38        39        40        41        42        43        44  \
0     -0.607174 -0.318885  0.472

In [9]:
X_test=preprocessor.transform(X_test)
X_test


array([[ 0.        ,  1.        ,  0.        , ...,  1.4223984 ,
         1.15591409,  3.09805664],
       [ 1.        ,  0.        ,  0.        , ..., -0.74024501,
        -0.80188714, -0.55782165],
       [ 1.        ,  0.        ,  0.        , ..., -0.90049315,
        -0.63197522, -0.8445572 ],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -0.23717736,
        -0.33528832, -0.24719147],
       [ 1.        ,  0.        ,  0.        , ..., -0.63423344,
         1.14462535,  0.7085937 ],
       [ 1.        ,  0.        ,  0.        , ..., -0.13727819,
        -0.38733392, -0.74897869]])

In [15]:
print(X_train.shape)
print(X_test.shape)

(24352, 47)
(6088, 47)


In [16]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae, rmse, r2_score

In [12]:
models={
    "Logisitic Regression":LogisticRegression(),
    "Decision Tree":DecisionTreeClassifier(),
    "Random Forest":RandomForestClassifier(),
    "Gradient Boost":GradientBoostingClassifier(),
    "Adaboost":AdaBoostClassifier(),
    "Xgboost":XGBClassifier()
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training set performance
    model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
    model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
    model_train_precision = precision_score(y_train, y_train_pred) # Calculate Precision
    model_train_recall = recall_score(y_train, y_train_pred) # Calculate Recall
    model_train_rocauc_score = roc_auc_score(y_train, y_train_pred)

    # Test set performance
    model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
    model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
    model_test_precision = precision_score(y_test, y_test_pred) # Calculate Precision
    model_test_recall = recall_score(y_test, y_test_pred) # Calculate Recall
    model_test_rocauc_score = roc_auc_score(y_test, y_test_pred) #Calculate Roc

    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print('- F1 score: {:.4f}'.format(model_train_f1))
    
    print('- Precision: {:.4f}'.format(model_train_precision))
    print('- Recall: {:.4f}'.format(model_train_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_train_rocauc_score))

    
    
    print('----------------------------------')
    
    print('Model performance for Test set')
    print('- Accuracy: {:.4f}'.format(model_test_accuracy))
    print('- F1 score: {:.4f}'.format(model_test_f1))
    print('- Precision: {:.4f}'.format(model_test_precision))
    print('- Recall: {:.4f}'.format(model_test_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_test_rocauc_score))

    print('='*35)
    print('\n')




Logisitic Regression
Model performance for Training set
- Accuracy: 0.9998
- F1 score: 0.9998
- Precision: 0.9996
- Recall: 1.0000
- Roc Auc Score: 0.9998
----------------------------------
Model performance for Test set
- Accuracy: 0.9997
- F1 score: 0.9997
- Precision: 0.9993
- Recall: 1.0000
- Roc Auc Score: 0.9997


Decision Tree
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 0.9951
- F1 score: 0.9951
- Precision: 0.9923
- Recall: 0.9977
- Roc Auc Score: 0.9951


Random Forest
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 0.9957
- F1 score: 0.9957
- Precision: 0.9933
- Recall: 0.9980
- Roc Auc Score: 0.9958


Gradient Boost
Model performance for Training se

#### Hyperparameter Tuning

In [31]:
LogisticReg_params={
    "penalty":['l1', 'l2', 'elasticnet'],
    "C":[100,10,1.0,0.1,0.01],
    "solver":['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

dt_param={
    'criterion':['gini','entropy', 'log_loss'],
    'splitter':['best','random'],
    'max_depth':[1,2,3,4,5],
    'max_features':['auto','sqrt','log2']
}
gradient_params={"loss": ['log_loss','deviance','exponential'],
             "criterion": ['friedman_mse','squared_error','mse'],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500],
              "max_depth": [5, 8, 15, None, 10]
                }


rf_params = {"max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500, 1000]}

adaboost_param={
    "n_estimators":[50,60,70,80,90],
    "algorithm":['SAMME','SAMME.R']
}
xgboost_params = {"learning_rate": [0.1, 0.01],
                  "max_depth": [5, 8, 12, 20, 30],
                  "n_estimators": [100, 200, 300],
                  "colsample_bytree": [0.5, 0.8, 1, 0.3, 0.4]}

In [32]:
GridSearchCV_models=[("LR", LogisticRegression(), LogisticReg_params)]
cv=StratifiedKFold()
model_param={}
for name, model, params in GridSearchCV_models:
    grid=GridSearchCV(estimator=model,param_grid=params,scoring='accuracy',cv=cv,n_jobs=-1)
    grid.fit(X_train, y_train)
    model_param[name] = grid.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])



c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
200 fits failed out of a total of 375.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1

---------------- Best Params for LR -------------------
{'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}


In [33]:
randomcv_models = [
                   ("DT", DecisionTreeClassifier(), dt_param),
                   ("RF", RandomForestClassifier(), rf_params),
                   ("GB", GradientBoostingClassifier(), gradient_params),
                   ("AB", AdaBoostClassifier(), adaboost_param),
                   ("Xgboost", XGBClassifier(), xgboost_params)
                   ]

In [34]:
model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    

Fitting 3 folds for each of 90 candidates, totalling 270 fits


c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 90 is smaller than n_iter=100. Running 90 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
90 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
61 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\PALAK GUPTA\anaconda3\Lib\site-pac

Fitting 3 folds for each of 100 candidates, totalling 300 fits


c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
72 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
29 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\PALAK GUPTA\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\PALAK GUP

Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 

In [ ]:
print(model_param[model_name])

In [ ]:
models={
    "LogisticRegression": LogisticRegression(C=100, penalty='l2', solver="newton-cg"),

    "DecisionTreeClassifier" : DecisionTreeClassifier(),

    "Random Forest":RandomForestClassifier(n_estimators=1000,min_samples_split=2,
                                          max_features=7,max_depth=None),
    "Gradient Boos" : GradientBoostingClassifier(),

    "Adaboost":AdaBoostClassifier(n_estimators=80, algorithm='SAMME'),
    "XGboost" : XGBClassifier()
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training set performance
    model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
    model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
    model_train_precision = precision_score(y_train, y_train_pred) # Calculate Precision
    model_train_recall = recall_score(y_train, y_train_pred) # Calculate Recall
    model_train_rocauc_score = roc_auc_score(y_train, y_train_pred)


    # Test set performance
    model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
    model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
    model_test_precision = precision_score(y_test, y_test_pred) # Calculate Precision
    model_test_recall = recall_score(y_test, y_test_pred) # Calculate Recall
    model_test_rocauc_score = roc_auc_score(y_test, y_test_pred) #Calculate Roc


    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print('- F1 score: {:.4f}'.format(model_train_f1))
    
    print('- Precision: {:.4f}'.format(model_train_precision))
    print('- Recall: {:.4f}'.format(model_train_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_train_rocauc_score))

    
    
    print('----------------------------------')
    
    print('Model performance for Test set')
    print('- Accuracy: {:.4f}'.format(model_test_accuracy))
    print('- F1 score: {:.4f}'.format(model_test_f1))
    print('- Precision: {:.4f}'.format(model_test_precision))
    print('- Recall: {:.4f}'.format(model_test_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_test_rocauc_score))

    
    print('='*35)
    print('\n')

In [ ]:
## Plot ROC AUC Curve
from sklearn.metrics import roc_auc_score,roc_curve
plt.figure()

# Add the models to the list that you want to view on the ROC plot
auc_models = [
{
    'label': 'Xgboost',
    'model':XGBClassifier(n_estimators=200,max_depth=12,learning_rate=0.1,
                           colsample_bytree=1),
    'auc':  0.8882
},
    
]
# create loop through all model
for algo in auc_models:
    model = algo['model'] # select the model
    model.fit(X_train, y_train) # train the model
# Compute False postive rate, and True positive rate
    fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
# Calculate Area under the curve to display on the plot
    plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (algo['label'], algo['auc']))
# Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig("auc.png")
plt.show() 